<a href="https://colab.research.google.com/github/daniel-falk/ai-ml-principles-exercises/blob/main/ML-training/intro-to-libraries/intro_to_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenCV is a python library for images
OpenCV can be used for:
* Drawing on images
* Performing common transformations
* Analyzing images
* Reading and writing images and video
* Converting between color spaces

The library and project is called `OpenCV` but the python package is named `cv2`.

In [ ]:
import cv2
from PIL import Image

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/8/8a/Automation_of_foundry_with_robot.jpg
bgr_img = cv2.imread("Automation_of_foundry_with_robot.jpg")

In [ ]:
print("Image type: ", type(bgr_img))
print("Image shape: ", bgr_img.shape)

In [ ]:
Image.fromarray(bgr_img)

In [ ]:
rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
Image.fromarray(rgb_img)

In [ ]:
Image.fromarray(cv2.resize(rgb_img, (50, 50)))

## Drawing
OpenCV can be used to draw text and shapes on images.

In [ ]:
rgb_draw = rgb_img.copy()
cv2.rectangle(rgb_draw, (150,50), (370,390), color=(255,0,0), thickness=3)
Image.fromarray(rgb_draw)

In [ ]:
rgb_draw = rgb_img.copy()
cv2.circle(rgb_draw, center=(175,240), radius=50, color=(0,255,0), thickness=3)
Image.fromarray(rgb_draw)

In [ ]:
rgb_draw = rgb_img.copy()
cv2.putText(rgb_draw, "Image 0", org=(50, 50), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=3, color=(255,0,0))
Image.fromarray(rgb_draw)

# Transforming images
Images can be transformed in various ways using the library.

In [ ]:
h, w = rgb_img.shape[:2]
center = (w//2,h//2)
  
# Generating a rotation matrix
angle = 45.
scale = 1.
rot_matrix = cv2.getRotationMatrix2D(center, angle, scale) 
  
# Performing the affine transformation
# Note that this is not an inplace operation since destination size
# might be different from the source size to not loose the corners of the image
rot_rgb = cv2.warpAffine(rgb_img, rot_matrix, dsize=(w, h))
Image.fromarray(rot_rgb)

In [ ]:
blur_rgb = cv2.GaussianBlur(rgb_img, ksize=(21, 21), sigmaX=9)
Image.fromarray(blur_rgb)

In [ ]:
edges_rgb = cv2.Canny(image=rgb_img, threshold1=100, threshold2=200)
Image.fromarray(edges_rgb)

In [ ]:
gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
blur_rgb = cv2.GaussianBlur(gray_img, ksize=(11, 11), sigmaX=2)
edges_rgb = cv2.Canny(image=blur_rgb, threshold1=20, threshold2=100)
Image.fromarray(edges_rgb)

# Object detection and matching
OpenCV does also have more complex algorithms such as face detectors and feature point matching.

In [ ]:
!wget "https://upload.wikimedia.org/wikipedia/commons/2/2e/Al_Hurricane_and_Al_Hurricane%2C_Jr._performing_at_the_San_Felipe_De_Neri_2014_fiestas.jpg" -O img.jpg
concert_img = cv2.cvtColor(cv2.imread("img.jpg"), cv2.COLOR_BGR2RGB)
h, w = concert_img.shape[:2]
concert_img = cv2.resize(concert_img, (w//4, h//4))
Image.fromarray(concert_img)

In [ ]:
# Download a pretrained cascade definition for the detector
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
concert_gray = cv2.cvtColor(concert_img, cv2.COLOR_RGB2GRAY)
faces = face_detector.detectMultiScale(concert_gray,  scaleFactor=1.1, minNeighbors=1)

# Draw the rectangle around each face
print(f"Found {len(faces)} faces")
draw_rgb = concert_img.copy()
for (x, y, w, h) in faces:
  cv2.rectangle(draw_rgb, (x, y), (x+w, y+h), color=(255, 0, 0), thickness=2)
Image.fromarray(draw_rgb)

# Feature point matching
Finding and matching features can be used to locate the same objects in similar images, or as in this example overlapping crops of the same image.

In [ ]:
from IPython.display import display

h, w = concert_img.shape[:2]
img1 = concert_img[:,:w//2 + 50]
img2 = concert_img[:,w//2 - 50:]
display(Image.fromarray(img1), Image.fromarray(img2))

In [ ]:
# Find features in the two images
feature = cv2.ORB_create(nfeatures=5000)
keypoints1, descriptors1 = feature.detectAndCompute(img1, None)
keypoints2, descriptors2 = feature.detectAndCompute(img2, None)

# match the features across the two images
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(descriptors1, descriptors2)
matches = sorted(matches, key=lambda x: x.distance)  # Sort most similar first

# draw first 50 matches
match_img = cv2.drawMatches(
    img1, keypoints1, img2, keypoints2,
    matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
Image.fromarray(match_img)